# Graph Theory Semester Project
*Submitted To:* Mr. Waqas Ali  
*Submitted By:*  

- 2021-CS-58 Mukarram Ali
- 2021-CS-59 Rayan Rasheed  



In [ ]:
!pip install networkx
!pip install nltk
!pip install scikit-learn
!pip install pandas
!pip install numpy
!pip install medium_api

Import Libraries

In [ ]:
import nltk
import os
import networkx as nx
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Scrape Data From World Wide Web

In [ ]:
!python Scraper.py


^C


Organizing the Scraping Data For Training

In [ ]:
folders = ["Fashion and Beauty_data", "Health and Fitness_data", "Travel_data"]
all_data = []
def read_text_files(folder_path, folder_name, max_files=15):
    file_data = []
    serial=1
    category=folder_name.replace("_data","")
    for idx, filename in enumerate(os.listdir(folder_path)):
        if idx >= max_files:
            break
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, "r", encoding="utf-8") as file:
                text = file.read()
                file_data.append((serial, text, category))
                serial+=1
    return file_data
for folder in folders:
    folder_path = os.path.join(os.getcwd(), folder)
    folder_data = read_text_files(folder_path, folder)
    all_data.extend(folder_data)

# Create DataFrame
df = pd.DataFrame(all_data, columns=["Serial Number", "Text Data", "Category"])

# Write DataFrame to Excel
output_excel_path = "Data.xlsx"
df.to_excel(output_excel_path, index=False)

Functions for MCS, Distance Matrix, Preprocess Data

In [ ]:

# Preprocess text
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    # Remove stop words and non-alphabetic tokens
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    # Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    return stemmed_tokens

# Construct document graph
def construct_document_graph(text):
    tokens = preprocess_text(text)
    G = nx.DiGraph()
    for token in set(tokens):
        G.add_node(token)
    for i in range(len(tokens) - 1):
        word1 = tokens[i]
        word2 = tokens[i + 1]
        if not G.has_edge(word1, word2):
            G.add_edge(word1, word2)
    return G

# Compute maximal common subgraph (MCS)
def maximal_common_subgraph(graph1, graph2):
    matcher = nx.algorithms.isomorphism.GraphMatcher(graph1, graph2)
    if matcher.subgraph_is_isomorphic():
        isomorphisms = next(matcher.subgraph_isomorphisms_iter())
        mcs = nx.Graph(graph1.subgraph(isomorphisms))
        return mcs
    else:
        return nx.Graph()  # Return an empty graph if no MCS exists

# Calculate distance measure based on MCS
def mcs_distance(graph1, graph2):
    num_vertices_graph1 = len(graph1.nodes)
    num_edges_graph1 = len(graph1.edges)
    num_vertices_graph2 = len(graph2.nodes)
    num_edges_graph2 = len(graph2.edges)

    # Compute MCS
    mcs = maximal_common_subgraph(graph1, graph2)
    num_vertices_mcs = len(mcs.nodes)
    num_edges_mcs = len(mcs.edges)

    # Calculate distance measure (e.g., dissimilarity ratio)
    vertex_similarity = num_vertices_mcs / min(num_vertices_graph1, num_vertices_graph2)
    edge_similarity = num_edges_mcs / min(num_edges_graph1, num_edges_graph2)

    # You can combine vertex_similarity and edge_similarity to get an overall distance measure
    # For example, you could take the average or maximum of the two

    return 1 - (vertex_similarity + edge_similarity) / 2  # Return a dissimilarity measure
def getDistancematric(doc_graph, documents, max_features):
    distances = []
    for train_doc_graph, _ in documents:
        if train_doc_graph is not doc_graph:
            distance = mcs_distance(doc_graph, train_doc_graph)
            distances.append(distance)

    # Pad or truncate the feature vector to ensure consistent dimensionality
    padded_distances = distances[:max_features] + [0] * (max_features - len(distances))

    return padded_distances


def train_knn_classifier(documents, max_features, k=5):
    X = []
    y = []
    for doc_graph, label in documents:
        # Convert graph to feature vector using MCS distances
        distancematric = getDistancematric(doc_graph, documents, max_features)
        X.append(distancematric)
        y.append(label)

    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X, y)

    return knn


def test_knn_classifier(knn_classifier, test_doc_graph, documents,max_features):
    X_test = getDistancematric(test_doc_graph, documents,max_features)
    predicted_label = knn_classifier.predict([X_test])[0]
    return predicted_label





Constructing graph from Scraped Data

In [ ]:
file_path = 'Data.xlsx'
data = pd.read_excel(file_path)
page_data=data['Text Data'].tolist()
genre_data=data['Category'].tolist()
documents=[]
for i in range(len(page_data)-5):
  graph = construct_document_graph(page_data[i])
  documents.append((graph,genre_data[i]))

Check Document Format

In [ ]:
print(documents)

[(<networkx.classes.digraph.DiGraph object at 0x796829717250>, 'Fashion and Beauty'), (<networkx.classes.digraph.DiGraph object at 0x796828bedde0>, 'Fashion and Beauty'), (<networkx.classes.digraph.DiGraph object at 0x796828beddb0>, 'Fashion and Beauty'), (<networkx.classes.digraph.DiGraph object at 0x796828bedfc0>, 'Health and Fitness')]


Split Data for Training and Testing


In [ ]:
train_docs, test_docs = train_test_split(documents, test_size=0.2, random_state=42)

Training


In [ ]:
# Train k-NN classifier
max_features = 8
knn_classifier = train_knn_classifier(train_docs, max_features, k=3)

Testing


In [ ]:
predictions = []
true_labels = []
for test_graph, true_label in test_docs:
    predicted_label = test_knn_classifier(knn_classifier, test_graph, train_docs, max_features)
    predictions.append(predicted_label)
    true_labels.append(true_label)
print(predictions)
print(true_labels)

print(classification_report(true_labels, predictions))

['Fashion and Beauty']
['Fashion and Beauty']
                    precision    recall  f1-score   support

Fashion and Beauty       1.00      1.00      1.00         1

          accuracy                           1.00         1
         macro avg       1.00      1.00      1.00         1
      weighted avg       1.00      1.00      1.00         1

